# Import relevant libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf


In [2]:
from keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# tf.config.experimental_run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode(True)
tf.data.experimental.enable_debug_mode(True)
tf.config.run_functions_eagerly(True)

TypeError: enable_debug_mode() takes 0 positional arguments but 1 was given

In [ ]:
img_dir = "/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set"

In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 150
input_shape = (150,150,1)

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)


In [ ]:
train_gen = data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                        batch_size= BATCH_SIZE,
                                        color_mode = "grayscale",
                                        shuffle=True,
                                        class_mode = "binary",
                                        subset= "training")

In [ ]:
val_gen = data_gen.flow_from_directory(img_dir,
                                      target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                      batch_size=BATCH_SIZE,
                                      color_mode = "grayscale",
                                      shuffle=False,
                                       class_mode = "binary",
                                       subset="validation"
                                      )

In [ ]:
labels = train_gen.class_indices
classes = list(labels.keys())

print(classes)

## Modelling

For the modelling purposes we will use a convolutional neural network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dropout

In [ ]:
model =  Sequential()

model.add(keras.layers.InputLayer(input_shape = (150,150,1)))
model.add(Conv2D(16,(3,3), activation = "relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32,(3,3), activation = "relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.compile(optimizer="adam",
             loss = "binary_crossentropy",
             metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):
    history = model.fit(train_gen,
                        verbose = 1,
                        epochs=5,
                        validation_data= val_gen,
                        steps_per_epoch= 3681//64,
                        validation_steps = 919//64)

In [ ]:
test_loss, test_acc =  model.evaluate(val_gen)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, "y", label = "Training loss")
plt.plot(epochs, val_loss, 'r', label = "Validation loss")

plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')

plt.legend()
plt.show()


acc =  history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, 'y', label = "Training acc")
plt.plot(epochs, val_acc, 'r', label = "Validation acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.legend()
plt.show()

In [ ]:
# model.save("model.h5")
model.save("model.keras")

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image, ImageOps

data = np.ndarray(shape = (1,150,150,1), dtype = np.float32)
image= Image.open(f'/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set/Healthy/Not Cancer  (39).jpg')
#image = Image.open(f'/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (25).tif')
size = (150,150)
image = ImageOps.grayscale(image)
image = ImageOps.fit(image, size)
image_array = np.asarray(image)
display(image)

# normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data = image_array.reshape((-1,150,150,1))

#data[0] = normalized_image_array

In [ ]:
prediction =  model.predict(data)
print(prediction[0][0])